## Main Goal 

Create a RAG pipeline that will accept a user query, retrieve the relevent data from a vector database, and finally, generate a useful response using with the help of an LLM.

## My Appraoch

At a high level, the goal can be broken down into a few parts,

- Processing the data (cleaning, creating chucks, etc)
- Vectorizing the data with  an embedding function
- Taking a query
- Retreiving relevant info using the vector database
- Feeding that info, plus the original query into the LLM and generating a response



In [1]:
import fitz  # PyMuPDF
import re
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from sentence_transformers import SentenceTransformer
import chromadb
from typing import List, Dict
import torch

In [7]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
path_to_book = "book/HSC26-Bangla1st-Paper.pdf"

In [12]:
def load_and_chunk_pdf(pdf_path: str, chunk_size: int = 500, chunk_overlap: int = 80):
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()
    
    full_text = "\n".join([page.page_content for page in pages])
    
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n"],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len
    )
    
    chunks = text_splitter.create_documents([full_text])
    
    return chunks


In [14]:
chunks = load_and_chunk_pdf(path_to_book)

In [16]:
chunks[0]

Document(metadata={}, page_content='অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,\nঅপরিরিতা\nআল ািয রিষয়\nিাাং া\n১ম পত্র\n১। অনুপলেি িািা কী কলি জীরিকা রনিবাহ কিলতন?\nক) ডাক্তার্ি খ) ওকালর্ত গ) মাস্টার্ি ঘ) ব্যব্সা\n২। োোলক ভাগ্য দেিতাি প্রধান এলজন্ট ি াি কািণ, তাি-\nক) প্রর্তপজি খ) প্রভাব্  গ) র্ব্চক্ষণতা ঘ) কূট ব্ুর্ি\nর্নকচি অনুকেদটি পক়ে ৩ ও ৪ সংখযক প্রকেি উিি দাও।\nর্পতৃহীন দীপুি চাচাই র্িকলন পর্িব্াকিি কতিা। দীপু র্িজক্ষত হকলও তাি র্সিান্ত যনও াি ক্ষমতা র্িল না। চাচা')

In [18]:
import pdfplumber

def load_pdf_with_pdfplumber(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

full_text = load_pdf_with_pdfplumber(path_to_book)

In [19]:
full_text[0:100]

'িাাং া\n১ম পত্র\nআল ািয রিষয়\nঅপরিরিতা\nঅনলাইন ব্যাচ সম্পর্কিত যেককাকনা জিজ্ঞাাসা ,\nর্িখনফল\n✓ র্নম্নর্ব্'

In [21]:
from paddleocr import PaddleOCR

ocr = PaddleOCR(use_angle_cls=True, lang='bn')  # Bangla support

def extract_text_with_ocr(pdf_path):
    # Convert PDF to images first (use pdf2image)
    images = convert_from_path(pdf_path)
    text = ""
    for img in images:
        result = ocr.ocr(img)
        text += " ".join([line[1][0] for line in result]) + "\n"
    return text

ValueError: No models are available for the language 'bn' and OCR version None.

In [22]:
import fitz  # PyMuPDF
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_pdf_with_fitz(pdf_path: str) -> str:
    """Extract text while preserving Bengali layout"""
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text", sort=True)  # 'sort=True' maintains reading order
    return text

def load_and_chunk_pdf(pdf_path: str, chunk_size: int = 800, chunk_overlap: int = 100):
    # 1. Extract with PyMuPDF
    raw_text = load_pdf_with_fitz(pdf_path)
    
    # 2. Bengali-specific cleaning
    cleaned_text = clean_bengali_text(raw_text)
    
    # 3. Semantic chunking
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n"],  # Bangla + English punctuation
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        is_separator_regex=False  # Required for non-ASCII separators
    )
    chunks = text_splitter.create_documents([cleaned_text])
    
    return chunks

In [23]:
texts = load_pdf_with_fitz(path_to_book)

In [27]:
print(texts[100:200])

জ্ঞাাসা  ,   র্িখনফল

  ✓র্নম্নর্ব্িব্যজক্তিহঠাৎর্ব্িিালীহক  ওঠািফকলসমাকিপর্িচ  সংকটসম্পককিধািণালাভক


In [28]:
def load_pdf_with_fitz(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        # Critical: Use 'text' mode with layout preservation
        text += page.get_text("text", sort=True, flags=fitz.TEXT_PRESERVE_LIGATURES | fitz.TEXT_PRESERVE_WHITESPACE)
    return text

In [30]:
texts = load_pdf_with_fitz(path_to_book)
print(texts[0:100])

    িাাং   া
   ১ম পত্র



       আল  ািয রিষয়

    অপরিরিতা





অনলাইন ব্যাচ সম্পর্কিত যেককাকনা জি


In [31]:
def clean_bengali_text(text: str) -> str:
    import re
    # Fix broken conjuncts and virama
    fixes = {
        r'্র্(\w)': r'্র\1',    # Fixes "র্খ" → "্রখ"
        r'্্': '্',            # Double virama → single
        r'|\✓|�': '',         # Remove garbage symbols
        r'(\p{L})্(\p{L})': r'\1\2্',  # Move virama to proper position
    }
    
    for pattern, replacement in fixes.items():
        text = re.sub(pattern, replacement, text)
    
    # Normalize Unicode composition
    import unicodedata
    text = unicodedata.normalize('NFC', text)
    
    return text

In [34]:
texts = clean_bengali_text(texts)
print(texts[0:100])

error: bad escape \p at position 1

In [36]:
import fitz  # PyMuPDF
from pdf2image import convert_from_path
from paddleocr import PaddleOCR
import numpy as np
import re

# Initialize PaddleOCR for Bengali
ocr_engine = PaddleOCR(use_angle_cls=True, lang='bn', show_log=False)

def is_bengali_text_valid(text: str) -> bool:
    """Check if extracted text contains valid Bengali"""
    bengali_chars = re.compile(r'[\u0980-\u09FF]')
    return len(bengali_chars.findall(text)) / max(1, len(text)) > 0.5  # >50% Bengali chars

def extract_with_ocr(page_image):
    """Process one page image with OCR"""
    result = ocr_engine.ocr(np.array(page_image))
    return " ".join([line[1][0] for line in result[0]])

def load_pdf_with_fallback(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    full_text = ""
    
    for page_num, page in enumerate(doc):
        # First try PyMuPDF extraction
        text = page.get_text("text", sort=True, flags=fitz.TEXT_PRESERVE_LIGATURES)
        
        # Validate Bengali text quality
        if not text or not is_bengali_text_valid(text):
            print(f"Falling back to OCR for page {page_num+1}")
            img = convert_from_path(pdf_path, first_page=page_num+1, last_page=page_num+1)[0]
            text = extract_with_ocr(img)
        
        full_text += text + "\n"
    
    return full_text

# Usage:
clean_text = load_pdf_with_fallback(path_to_book)

ValueError: No models are available for the language 'bn' and OCR version None.

In [37]:
import pytesseract
from pdf2image import convert_from_path

def ocr_with_tesseract(pdf_path):
    images = convert_from_path(pdf_path, dpi=300)
    text = ""
    for img in images:
        text += pytesseract.image_to_string(img, lang='ben') + "\n"
    return text



In [38]:
text = ocr_with_tesseract(path_to_book)


'10940759\n\nঅনলাইন ব্যাট”\n\nহি\nবাংলা * ইংরেজি * আইসিটি\n\n \n\n  \n\nঅনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়'

In [39]:
text[100:200]

',\n\n01550188810,\n\x0c\n[লুল\nজআললাইন ব্যাচ” 11072\n1 ১4159?\n\n/ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার'

In [40]:
def save_text_to_file(text: str, file_path: str) -> None:
    with open(file_path, 'w', encoding='utf-8') as f:
        f.write(text)

In [42]:
save_text_to_file(text, "data/unclean_text.txt")

In [60]:
import re

## this is good and much much less aggressive compared to prev ones
def precise_bengali_cleaner(text: str) -> str:
    """
    Targeted cleaning for Bengali OCR text:
    - Removes exactly the garbage patterns shown in your examples
    - Preserves all other Bengali text and numbers
    """
    # Pattern 1: Form feed followed by bracket garbage (e.g., '[লুল')
    text = re.sub(r'\x0c\[[^\]]+\]', '', text)
    
    # Pattern 2: Isolated number sequences (e.g., '11072', '1 ১4159?')
    text = re.sub(r'(?<!\S)\d[\d ১২৩৪৫৬৭৮৯০?]+\b', '', text)
    
    # Pattern 3: European symbols with numbers (e.g., '€91717110')
    text = re.sub(r'[€£][\d]+', '', text)
    
    # Remove remaining form feeds and trailing whitespace
    text = text.replace('\x0c', '').strip()
    
    # Preserve all Bengali characters, punctuation and meaningful numbers
    return text

def ultra_precise_cleaner(text: str) -> str:
    """
    Removes ONLY these exact sequences:
    1. '[লুল' at line start
    2. 'জআললাইন ব্যাচ”' 
    3. Isolated '?' on its own line
    Preserves all other content including newlines
    """
    # Pattern 1: '[লুল' at line start (keeps following newline)
    text = re.sub(r'^\[লুল\n', '', text, flags=re.MULTILINE)
    
    # Pattern 2: 'জআললাইন ব্যাচ”' (standalone)
    text = re.sub(r'জআললাইন ব্যাচ”\n?', '', text)
    
    # Pattern 3: Isolated '?' on its own line
    text = re.sub(r'^\?$\n', '', text, flags=re.MULTILINE)
    
    return text

In [65]:
clean_text = precise_bengali_cleaner(text)
clean_text = ultra_precise_cleaner(clean_text)
save_text_to_file(clean_text, "data/clean_text.txt")

In [66]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load your cleaned text
with open("data/clean_text.txt", "r", encoding="utf-8") as f:
    cleaned_text = f.read()

# Configure the splitter for Bengali text
bengali_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"],  # Bengali-relevant separators
    chunk_size=1000,  # Adjust based on your needs
    chunk_overlap=200,  # Preserves context across chunks
    length_function=len,
    is_separator_regex=False  # Important for non-ASCII separators
)

# Split the documents
text_chunks = bengali_splitter.create_documents([cleaned_text])

# Optional: Verify the chunks
for i, chunk in enumerate(text_chunks[:3]):  # Print first 3 chunks
    print(f"Chunk {i+1}:")
    print(chunk.page_content)
    print("\n" + "="*50 + "\n")


Chunk 1:
অনলাইন ব্যাট”

হি
বাংলা * ইংরেজি * আইসিটি

 

  

অনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,

,

 

/ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।
তৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।
তৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।

 

তৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে৷
নারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে
পারবে।
মানুষ আশা নিয়ে বেঁচে থাকে- অনুপমের দৃষ্টান্তে মানবজীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে
জ্ঞানলাভ করবে।
ছ প্রাক-মূল্যায়ন
১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?
ক) ডাক্তারি খ) ওকালতি গ) মাস্টারি ঘ) ব্যবসা
২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-
ক) প্রতিপত্তি খ) প্রভাব গ) বিচক্ষণতা ঘ) কুট বুদ্ধি

নিচের অনুচ্ছেদটি পড়ে ও সংখ্যক প্রশ্নের উত্তর দাও।


Chunk 2:
নিচের অনুচ্ছেদটি পড়ে ও সংখ্যক প্রশ্নের উত্তর দাও।

পিতৃহীন দীপুর চাচাই ছিলেন পরিবারের কর্তা। দীপু শিক্ষিত হলেও তার সিদ্ধান্ত নেওয়ার ক্ষমতা 

In [82]:
def text_splitter(txt_path : str):
    with open(txt_path, "r", encoding="utf-8") as f:
        cleaned_text = f.read()
    
    # Configure the splitter for Bengali text
    bengali_splitter = RecursiveCharacterTextSplitter(
        # separators=["\n\n", "\n"],  # Bengali-relevant separators
        chunk_size=1000,  # Adjust based on your needs
        chunk_overlap=100,  # Preserves context across chunks
        length_function=len,
        is_separator_regex=False  # Important for non-ASCII separators
    )
    
    # Split the documents
    text_chunks = bengali_splitter.create_documents([cleaned_text])
    return text_chunks

In [83]:
chuckz = text_splitter("data/clean_text.txt")

In [84]:
chuckz[0]

Document(metadata={}, page_content='অনলাইন ব্যাট”\n\nহি\nবাংলা * ইংরেজি * আইসিটি\n\n \n\n  \n\nঅনলাইন ব্যাচ সম্পর্কিত যেকোনো জিজ্ঞাসায়,\n\n,\n\n \n\n/ নিম্নবিত্ত ব্যক্তির হঠাৎ বিত্তশালী হয়ে ওঠার ফলে সমাজে পরিচয় সংকট সম্পর্কে ধারণা লাভ করবে।\nতৎকালীন সমাজ-সভ্যতা ও মানবতার অবমাননা সম্পর্কে জানতে পারবে।\nতৎকালীন সমাজের পণপ্রথার কুপ্রভাব সম্পর্কে জানতে পারবে।\n\n \n\nতৎকালে সমাজে ভদ্রলোকের স্বভাববৈশিষ্ট্য সম্পর্কে জ্ঞানলাভ করবে৷\nনারী কোমল ঠিক, কিন্তু দুর্বল নয়- কল্যাণীর জীবনচরিত দ্বারা প্রতিষ্ঠিত এই সত্য অনুধাবন করতে\nপারবে।\nমানুষ আশা নিয়ে বেঁচে থাকে- অনুপমের দৃষ্টান্তে মানবজীবনের এই চিরন্তন সত্যদর্শন সম্পর্কে\nজ্ঞানলাভ করবে।\nছ প্রাক-মূল্যায়ন\n১। অনুপমের বাবা কী করে জীবিকা নির্বাহ করতেন?\nক) ডাক্তারি খ) ওকালতি গ) মাস্টারি ঘ) ব্যবসা\n২। মামাকে ভাগ্য দেবতার প্রধান এজেন্ট বলার কারণ, তার-\nক) প্রতিপত্তি খ) প্রভাব গ) বিচক্ষণতা ঘ) কুট বুদ্ধি\n\nনিচের অনুচ্ছেদটি পড়ে ও সংখ্যক প্রশ্নের উত্তর দাও।')

In [85]:
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={"device": "cpu"}  # or "cuda" for GPU
)

In [86]:
vector_db = Chroma.from_documents(
    documents=chuckz,
    embedding=embeddings,
    persist_directory="./bengali_chroma_db"  # Local storage
)

In [87]:
# Test with a Bengali query
# query = "অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে?"
# query = "বাংলা সাহিত্যের জনক কে?"
# query = "বাংলা সাহিত্যের জনক কে?"
query = "কাকে অনুপমের ভাগ্য দেবতা বলে উল্লেখ করা হয়েছে?"
results = vector_db.similarity_search(query, k=3)

for i, doc in enumerate(results):
    print(f"Result {i+1}:\n{doc.page_content}\n{'-'*50}")

Result 1:
বলা বাহুল্য, আমিও খুব রাগিয়াছিলাম। কোনো গতিকে শস্তুনাথ বিষম জব্দ হইয়া আমাদের পায়ে ধরিয়া
আসিয়া পড়েন, গোঁফের রেখায় তা দিতে দিতে এইটেই কেবল কামনা করিতে লাগিলাম।

কিন্ত, এই আক্রোশের কালো রঙের শ্োতের পাশাপাশি আর-একটা শ্রোত বহিতেছিল যেটার রঙ একেবারেই
কালো নয়। সমস্ত মন যে সেই অপরিচিতার পানে ছুটিয়া গিয়াছিল__এখনো যে তাহাকে কিছুতেই টানিয়া
ফিরাইতে পারি না। দেয়ালটকুর আড়ালে রহিয়া গেল গো। কপালে তার চন্দন আঁকা, গায়ে তার লাল শাড়ি,
মুখে তার লজ্জার রক্তিমা, হৃদয়ের ভিতরে কী যে তা কেমন করিয়া বলিব।

 

,


বাংলা - ইংরেজি *তআইসিটি

নত হইয়া পড়িয়াছিল। হাওয়া আসে, গন্ধ পাই,
পাতার শব্দ শুনি__ কেবল আর একটিমাত্র পা
ফেলার অপেক্ষা-_-এমন সময়ে সেই এক
পদক্ষেপের দুরত্বটুকু এক মুহর্তে অসীম হইয়া
উঠিল!
--------------------------------------------------
Result 2:
বলা বাহুল্য, আমিও খুব রাগিয়াছিলাম। কোনো গতিকে শস্তুনাথ বিষম জব্দ হইয়া আমাদের পায়ে ধরিয়া
আসিয়া পড়েন, গোঁফের রেখায় তা দিতে দিতে এইটেই কেবল কামনা করিতে লাগিলাম।

কিন্ত, এই আক্রোশের কালো রঙের শ্োতের পাশাপাশি আর-একটা শ্রোত বহিতেছ

In [80]:
print(len(set([doc.page_content for doc in chuckz])))

107


In [81]:
len(chuckz)

107